## 📏 Real-Time Object Measurement

### ✅ If the Camera Never Moves
If the **camera position, angle, focus, and distance stay the same**, then the measurement remains accurate **only if the new object is placed exactly where the calibrated object was**.

---

### ✅ When It **Will Still Be Accurate**
✔️ Object is on the **same flat surface**
✔️ Object is at the **same distance from the camera**
✔️ Object is **not tilted** (same orientation as the remote)
✔️ Camera does **not auto-zoom or change focal length**
✔️ Lighting does not break contour detection

🔹 In this scenario, replacing the remote with an iPhone will still produce correct measurement.

---

### ❌ When It Becomes Inaccurate (Even if Camera Doesn’t Move)
| Issue                                 | Why it breaks                     |
| ------------------------------------- | --------------------------------- |
| Object is closer/farther from camera  | Pixel size changes                |
| Object is tilted upward/downward      | Perspective distortion            |
| Camera autofocus changes focal length | Scale becomes invalid             |
| Different edge detection shape        | Bounding box changes              |
| Object is thick or curved             | Detected contour is not true size |

---

### 📌 Key Reminder
Your current method uses **2D pixel-to-cm scaling** based on a **single object calibration**.
It assumes the object lies on **the same plane (same Z-depth)**.

```
cm_per_pixel = real_size_cm / detected_pixel_size
```

If depth changes → pixel size changes → result becomes wrong.

---

### 🔍 Example
| Object               | Distance | Pixel Height | Real Height | Result             |
| -------------------- | -------- | ------------ | ----------- | ------------------ |
| Remote (calibration) | 40 cm    | 250 px       | 16 cm       | ✅ Correct          |
| iPhone in same spot  | 40 cm    | 400 px       | 14.7 cm     | ✅ Correct          |
| iPhone moved closer  | 35 cm    | 450 px       | 14.7 cm     | ❌ Measured too big |

---

### ✅ Final Answer
* ✅ If the new object is in **exact same position** → measurement stays accurate.
* ❌ If distance or angle changes even slightly → measurement becomes inaccurate.

---

### 🛠 Want Accuracy for Any Object, Any Distance?
| Method                                   | Depth Safe? | Works for Any Object? |
| ---------------------------------------- | ----------- | --------------------- |
| Use reference object (coin, credit card) | ✅           | ✅                     |
| Use ArUco marker                         | ✅           | ✅                     |
| Stereo / depth camera                    | ✅           | ✅                     |
| Full camera calibration                  | ✅           | ✅                     |
| Your current single-object calibration   | ❌           | ❌                     |

## Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# DISPLAY FUNCTION (unchanged from your original)
def display_comparison(
    original_img, edge_img, title1="Edge Detection", title2="Detected Object"
):
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))
    axs[0].imshow(cv2.cvtColor(edge_img, cv2.COLOR_BGR2RGB))
    axs[0].set_title(title1)
    axs[0].axis("off")
    axs[1].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    axs[1].set_title(title2)
    axs[1].axis("off")
    plt.tight_layout()
    plt.show()


# SELECT IMAGE OR VIDEO (Real Time -> Webcame=True | Detect Video -> Webcam = False)
use_webcam = True
video_path = "assets/video1.mov"

if use_webcam:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Tidak bisa membuka video / webcam.")


# REAL SIZE REFERENCE
actual_width_cm = 3.5
actual_height_cm = 16.0


# PROCESS EACH FRAME
while True:
    ret, img = cap.read()
    if not ret:
        break

    frame = img.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Removed object mask and brightness/contrast adjustment for consistency
    # (they were causing inconsistent processing between iPhone and remote)
    # 3) Edge detection pipeline (simplified, now on original gray)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)
    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # 4) Contour detection
    contours, _ = cv2.findContours(
        thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    if contours:
        largest = max(contours, key=cv2.contourArea)

        # FIXED MEASUREMENT (minAreaRect - angle independent)
        rect = cv2.minAreaRect(largest)
        (center_x, center_y), (w_rot, h_rot), angle = rect

        # Sort width/height so long side matches real object height
        detected_width_px = min(w_rot, h_rot)
        detected_height_px = max(w_rot, h_rot)

        #  Compute cm per pixel (accurate)
        cm_per_pixel_w = actual_width_cm / detected_width_px
        cm_per_pixel_h = actual_height_cm / detected_height_px
        cm_per_pixel_avg = (cm_per_pixel_w + cm_per_pixel_h) / 2

        #  Convert detected object size
        object_width_cm = detected_width_px * cm_per_pixel_avg
        object_height_cm = detected_height_px * cm_per_pixel_avg

        #  Draw rotated bounding box
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)

        cv2.putText(
            frame,
            f"Width: {object_width_cm:.2f} cm",
            (box[0][0], box[0][1] - 25),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2,
        )

        cv2.putText(
            frame,
            f"Height: {object_height_cm:.2f} cm",
            (box[0][0], box[0][1] - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2,
        )

    cv2.imshow("Accurate Object Measurement (Video)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_21180\2352781231.py:81: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
